In [7]:
import sys
import os
sys.path.append("engine/")
sys.path.append("funcs/")
sys.setrecursionlimit(10000)

import theano.gpuarray
theano.gpuarray.use('cuda0')

import theano
import theano.tensor as T
from theano import config
import numpy as np
import pickle
import copy
import time
from itertools import *

from matplotlib import pyplot as plt
from matplotlib import gridspec
%matplotlib inline

from VIMCO_alpha import *

import scipy.io

Mapped name None to device cuda0: Tesla V100-PCIE-32GB (0000:65:02.0)


In [2]:
config.gpu_preallcoate = 0.9

In [63]:
traces_prep = [[] for i in range(5)]
for i in range(5):
    datay = scipy.io.loadmat('/root/autodl-tmp/DeepSpike-master/dataset/crcns_cai-3_data/data.{}.train.preprocessed.mat'.format(i+1))
    # print('./crcns_cai-3_data/data.{}.train.preprocessed.mat'.format(i+1))
    print(len(datay['data'][0]))
    for j in range(len(datay['data'][0])):
        traces_prep[i].append(datay['data'][0][j]['calcium'][0][0][0])
traces_prep

11
21
13
6
9


[[array([-0.23492758, -0.18244086, -0.04925736, ...,  0.67032284,
          0.35596177, -0.03306312]),
  array([0.68918643, 0.86913181, 0.94927333, ..., 0.33546066, 0.39531902,
         0.51772934]),
  array([2.57856035, 2.86860137, 2.78498888, ..., 1.10403196, 1.43992057,
         2.01759083]),
  array([0.63676983, 0.59616368, 0.71187544, ..., 0.83873393, 0.84280121,
         0.75983993]),
  array([-0.01513953,  0.02246397,  0.04002773, ..., -0.26401131,
         -0.18623395, -0.08680878]),
  array([0.81977139, 0.83141094, 0.87719118, ..., 1.17769838, 1.02498992,
         0.87915102]),
  array([2.44210493, 2.9302013 , 3.06587224, ..., 0.21532572, 0.68898524,
         1.58323478]),
  array([0.39131044, 0.49021257, 0.61087252, ..., 0.15193403, 0.22134589,
         0.3072331 ]),
  array([0.22352287, 0.11609466, 0.1544367 , ..., 0.63148281, 0.54563117,
         0.396095  ]),
  array([1.55767456, 1.62805426, 1.47938152, ..., 0.3971513 , 0.74026   ,
         1.20988914]),
  array([0.6250518

In [64]:
data_dict = {'traces': traces_prep, 'spikes': Spikes_train, 'fps' : [50,50,50,50,50], 'spike_fps': 50}

with open('new_data_prepped.pkl', 'wb') as f:
    pickle.dump(data_dict, f, pickle.HIGHEST_PROTOCOL)

In [65]:
buffers = [100,40]
timebins_eval = 250
timebins = timebins_eval + sum(buffers)

samples = 64
lr = 5e-4
batch_size = 12

rng = np.random.RandomState(1)
lasagne.random.set_rng(rng)

# with open('data_prepped.pkl', 'rb') as f:
#     data = pickle.load(f)
    

with open('new_data_prepped.pkl', 'rb') as f:
    data = pickle.load(f)
n_cells = 5

Traces_train = data['traces'][:n_cells]
Spikes_train = data['spikes'][:n_cells]

fps = data['fps'][:n_cells]
# fps = [50,50,50,50,50]
spikefps = data['spike_fps']
# spikefps = 50

# sv_Traces,sv_Spikes = data_chop(timebins, Traces_train, Spikes_train, [],fps, spikefps,50, fb=False)
# sv_Spikes = [[] for _ in range(n_cells)]
# test_tr_us, test_sp_us = data_chop(1798, Traces_train,  Spikes_train, None, fps, spikefps,50,fb=False)

In [70]:
print(data['spike_fps'])
data['fps']

50


[50, 50, 50, 50, 50]

In [ ]:
def plot_preds_bl_cr_v2(model, data, cell=0, pred=True, ts=[0, 1000000], trace=0, figsize=None, GS=None, prob=0.25):
    # print('cr')
    n_samples = 13

    # if figsize is None:
    #     fig = plt.figure()
    # elif isinstance(figsize, tuple):
    #     fig = plt.figure(figsize=figsize)
    # else:
    #     fig = figsize

    cp = sns.hls_palette(11, l=.4, s=.8)
    try:
        sr = model.superres
    except KeyError:
        sr = 1

    start = 0
    # end = - 1
    end = len(data['spikes'][cell][trace])

    fluor, spikes = data_resamp(data['traces'], data['spikes'], data['fps'], data['spike_fps'], model.resample)
    fluor = np.array(fluor[cell][trace][ts[0]:ts[1]], ndmin=2).astype(config.floatX)
    spikes = np.array(spikes[cell][trace][sr * ts[0]:sr * ts[1]], ndmin=2).astype(config.floatX)

    # model.mgen.load_genparams(cell)
    model.mgen.load_genparams(0)
    rec_dict = model.mrec.get_sample(fluor, n_samples)
    pred_prob = rec_dict['Probs']
    # print(len(pred_prob[0]))
    pred_spikes = rec_dict['Spikes']
    pred_bl = 0

    
    dt = 1 / model.resample / sr
    c = 1
    t = np.arange(len(fluor[0, start:end])) * dt
    t_sr = np.arange(len(fluor[0, start:end]) * sr) * dt / sr
    
    if model.mrec.n_genparams:
        pred_pars = rec_dict['Params']
    
    # print(sr * start, sr * end, sr)
    # pred_prob = pred_prob[:, sr * start:sr * end:sr].mean(axis=0)
    pred_prob = pred_prob.mean(axis=0)
    
    
    # pred_prob = np.where(pred_prob > prob, pred_prob, 0)
    # pred_prob = np.where(pred_prob > prob, 1, 0)
    
    if prob==0:
        pred_prob = np.where(pred_prob > prob, pred_prob, 0)
    else:
        pred_prob = np.where(pred_prob > prob, 1, 0)
    
    TP = 0  # 预测结果中实际为真的样本数
    FN = 0
    TN = 0
    FP = 0
    for i in range(len(pred_prob)):        
        if spikes[0, i] == 0:
            if pred_prob[i] == 0:
                TN += 1  
            else:
                FP += 1
        else:
            if pred_prob[i] == 0:
                FN += 1
            else:
                TP += 1
    print("FP:",FP)
    print('TN:',TN)
    FPR = FP / (FP + TN) # 假阳性
    FNR = FN / (TP + FN) # 假阴性
    TPR = TP / (TP + FN) # 真阳性
    
    pre_rate = (TP + TN) / len(pred_prob)
    
    
    # return FPR, FNR, pre_rate
    print('假阳性：', FPR)
    print('假阴性：', FNR)
    print('真阳性：', TPR)
    print('准确率：', pre_rate)
    print(len(pred_prob))
    print(len(spikes[0, :]))
    
    
    
    return FPR, FNR, TPR, pre_rate

In [15]:
def smooth2nd(x,M): ##x 为一维数组
    K = round(M/2-0.1) ##M应为奇数，如果是偶数，则取大1的奇数
    lenX = len(x)
    if lenX<2*K+1:
        print('数据长度小于平滑点数')
    else:
        y = np.zeros(lenX)
        for NN in range(0,lenX,1):
            startInd = max([0,NN-K])
            endInd = min(NN+K+1,lenX)
            y[NN] = np.mean(x[startInd:endInd])
##    y[0]=x[0]       #首部保持一致
##    y[-1]=x[-1]     #尾部也保持一致
    return(y)

In [71]:
import copy
data_smooth = copy.deepcopy(data)
# Traces_train = data_smooth['traces'][:n_cells]
# Spikes_train = data_smooth['spikes'][:n_cells]

for i in range(n_cells):
    for j in range(len(Traces_train[i])):
        # print(len(Traces_train[i][j]))
        # Traces_train[i][j] = smooth2nd(Traces_train[i][j], 15)
        data_smooth['traces'][i][j] = smooth2nd(data_smooth['traces'][i][j], 30)

# cell0

In [90]:
# t_sr, pred_prob = plot_preds_bl_cr_v2(model_500,data,cell=0,trace=0,figsize=(35,7),ts=[0,2000], prob=0)

# cell0 smooth

In [9]:
FPR, FNR, TPR, pre_rate = plot_preds_bl_cr_v2(model_500,data_smooth,cell=0,trace=0,figsize=(35,7),ts=[0,200000], prob=0.12)

FP: 2175
TN: 14112
假阳性： 0.13354208878246454
假阴性： 0.3990450204638472
真阳性： 0.6009549795361528
准确率： 0.8445333183123979
17753
17753


# cell1

In [89]:
# t_sr, pred_prob = plot_preds_bl_cr_v2(model_500,data,cell=1,trace=0,figsize=(35,7),ts=[0,4000000],prob=0)

# cell1 smooth

In [69]:
# t_sr, pred_prob = plot_preds_bl_cr(model_500,data_smooth,cell=1,trace=0,figsize=(35,7),ts=[0,2000])

In [ ]:
# t_sr, pred_prob = plot_preds_bl_cr_v2(model_500,data_smooth,cell=1,trace=0,figsize=(35,7),ts=[0,600000],prob=0.12)

# cell2

In [91]:
# t_sr, pred_prob = plot_preds_bl_cr_v2(model_500,data,cell=2,trace=4,figsize=(35,7),ts=[0,6000], prob=0)

In [10]:
FPR, FNR, TPR, pre_rate = plot_preds_bl_cr_v2(model_500,data,cell=4,trace=4,figsize=(35,7),ts=[0,6000000], prob=0.12)

FP: 460
TN: 7661
假阳性： 0.05664327053318557
假阴性： 0.6627218934911243
真阳性： 0.33727810650887574
准确率： 0.9191393781770895
8459
8459


In [75]:
print('numbers of cell:', len(data_smooth['traces']))
for c in range(len(data_smooth['traces'])):
    print('Cell ', c)
    print('traces ', len(data_smooth['traces'][c]))
    for t in range(len(data_smooth['traces'][c])):
        print(len(data_smooth['traces'][c][t]))

numbers of cell: 5
Cell  0
traces  11
71986
71987
35993
71985
71986
71987
71986
71985
71986
59717
71986
Cell  1
traces  21
35508
63034
37082
54398
45599
33685
48929
45014
27414
48039
59274
32049
56189
56241
47032
38917
25709
56555
21252
31077
9682
Cell  2
traces  13
53229
60908
24401
20014
27003
16802
60957
60908
60908
60957
60908
60908
60908
Cell  3
traces  6
27763
33280
33229
32000
32973
31066
Cell  4
traces  9
16919
16919
16919
16919
16919
16919
16919
16919
16919


In [72]:
import copy
noise = copy.deepcopy(data['traces'])
snr = copy.deepcopy(data['traces'])
Traces_train_smooth = copy.deepcopy(data_smooth['traces'])

In [ ]:
import pandas as pd
print('numbers of cell:', len(data_smooth['traces']))
for c in range(len(data_smooth['traces'])):
# for c in range(4,5):
    print('Cell ', c)
    print('traces ', len(data_smooth['traces'][c]))
    t1 = time.time()
    output0 = ['traces']
    output1 = ['fpr']
    # output2 = ['假阴率']
    output3 = ['acc']
    output4 = ['snr']
    for t in range(len(data_smooth['traces'][c])):
        
        # plot_preds_bl_cr_v2(model_500,data_smooth,c,trace=t,figsize=(35,7),ts=[0,1400],prob=0)
        FPR, FNR, TPR, pre_rate = plot_preds_bl_cr_v2(model_500,data_smooth,c,trace=t,figsize=(35,7),ts=[0,100000],prob=0.12)
        output0.append(t)
        output1.append(round(FPR, 2))
        # output2.append(FNR)
        output3.append(round(pre_rate, 2))
        # plt.title('cell:{0} trace: {1}'.format(c, t))
        # plt.savefig('/root/autodl-tmp/DeepSpike-master/output2' + folder + '/cell:{0} trace: {1}.png'.format(c, t))
        # plt.show()
        # Traces_train_smooth[c][t] = smooth2nd(Traces_train[c][t], 25)
        noise[c][t] = data['traces'][c][t] - data_smooth['traces'][c][t]
        snr = 10 * np.log10(np.sum(data_smooth['traces'][c][t]**2) / np.sum(noise[c][t]**2))
        output4.append(round(snr, 2))
    output0 = pd.DataFrame(output0)
    output0 = output0.T
    output1 = pd.DataFrame(output1)
    output1 = output1.T
    # output2 = pd.DataFrame(output2)
    # output2 = output2.T
    output3 = pd.DataFrame(output3)
    output3 = output3.T
    output4 = pd.DataFrame(output4)
    output4 = output4.T
    output = pd.concat([output0,output1])
    # output = pd.concat([output,output2])
    output = pd.concat([output,output3])
    output = pd.concat([output,output4])
    
    
    # output = pd.DataFrame(output)
    # print(output)
    # output = output.T
    print(output)
    # output2 = pd.DataFrame(output2)
    # output = output.append(output3)
    # output.T.to_excel("./test.xlsx")
    # output.to_csv("./data.{}.train.preprocessed.csv".format(c+1), index=False)
    # output.T.to_excel('/root/autodl-tmp/DeepSpike-master/OUTPUT/output_part6/' + folder + '/' + path[c][67:], index = False, header=None)
    print(time.time()-t1)